# The Richardson Iteration

also called simple iteration is the fixed point iteration

$$
x^{k+1} := x^k + \alpha \, (b - A x^k)
$$

with an arbitrary starting value $x^0 \in {\mathbb R}^n$, and a properly cosen damping parameter $\alpha$. The misfit of the equation, $r = b - A x^k$ is called residual.

The solution $x^\ast$ is a fixed point of the iteration (since $b - A x^\ast = 0$).

If we define the error as

$$
e^k = x^k - x^\ast,
$$

the error propagation from one step to the next is

\begin{align*}
e^{k+1} & = x^{k+1} - x^\ast = x^k + \alpha (b - A x^k) - x^\ast \\
& = x^k - x^\ast + \alpha A (x^\ast - x^k) = (I - \alpha A) (x^k - x^\ast)
\end{align*}

This means the new error is obtained from the old error by the error propagation matrix

$$
e^{k+1} = \underbrace{(I - \alpha A)}_{M} \, e^k
$$

Two strategies to verify convergence are:

* prove that the spectral radius 

$$
\rho(I - \alpha A) := \max_{\lambda \in \sigma(I - \alpha A)} |\lambda| < 1
$$

* find some norm $\| \cdot \|$ such that the matrix norm (=operator norm)

$$
\| I - \alpha A \| := \sup_{x \in {\mathbb R}^n} \frac{ \| (I - \alpha A) x \| }{ \| x \| } < 1
$$

The first one, $\rho < 1$, only provides asymptotic convergence. This is easily proven if A is diagonizable, i.e. it features a full set of eigenvectors $z^j$ and eigenvalues $\lambda_j$. Expand the initial error as

$$
e^0 = \sum_j e^0_j z^j,
$$
then

$$
e^k = \sum_j (1-\alpha \lambda_j)^k e^0_j z^j
$$
and

$$
\| e^k \| \leq \rho^k  \sum_j | e^0_j | \| z^j \|
$$
This means $\| e^k \| \leq C \rho^k$, but the error does not have to decrease monotonically.

However, if $\| I - \alpha A \| < 1$, then

$$
\| e^{k+1} \| \leq \| I - \alpha A \| \, \| e^k \|,
$$
proves that the error decreases in every iteration step. Note that the matrix norm is the operator norm generated by the vector norm.

Some facts:
* If the norm $\| \cdot \|$ is generated by an inner product $\left< \cdot, \cdot \right>$ (parallelogram identity), and $M$ is some self adjoint matrix with respect to this inner product, i.e.
  
  $$
  \left< M x, y \right>  = \left< x, M y \right>, 
  $$
  then $\rho(M) = \| M \|$

*  If $\left< \cdot , \cdot \right>$ is the Euklidean inner product, then $M$ is self-adjoint exactly when $M$ is symmetric.
  
* The spectral radius is bounded by every operator norm. There exists some norm such that the operator norm is arbitrary close to the spectral radius, i.e.
  
  $$
  \rho(M) = \inf_{ \text{norms} \| \cdot \| } \| M \|
  $$

## Optimizing the relaxation parameter $\alpha$

Let $A$ be SPD, and let $\sigma(A) = \{ \lambda_i \in {\mathbb R} \}$ with $0 < \lambda_1 \leq \lambda_2 \ldots \leq \lambda_n$.

Then the eigenvalues of the error propagation matrix $M = I - \alpha A$ are $\{ 1 - \alpha \lambda_i  \}$. 

Whenever we choose 

$$
0 < \alpha < \frac{2}{\lambda_n}
$$
we obtain $\rho(M) < 1$, and a convergent iteration.


The spectral radius of $M$ is 

$$
\rho(M) = \max_i \{ | 1 - \alpha \lambda_i| \}  = 
\max \{ 1 - \alpha \lambda_1, - (1-\alpha \lambda_n) \}
$$

The maximum is minimized if we choose $\alpha$ optimally such that

$$
1 - \alpha \lambda_1 = - (1 - \alpha \lambda_n),
$$
i.e.

$$
\alpha_{\text{opt}} = \frac{2}{\lambda_1 + \lambda_n} 
$$
leading to the optimal rate of convergence

$$
\rho_{\text{opt}} = \frac{\lambda_n - \lambda_1}{\lambda_n+\lambda_1}
\approx 1 - 2 \frac{\lambda_1}{\lambda_n} = 1 - \frac{2}{\kappa(A)}
$$

with the condition number $\kappa(A) = \lambda_n(A) / \lambda_1(A)$.


After $N$ iterations, the error is reduced by 

$$
\left(1 - \frac{1}{\kappa}\right)^N.
$$

To obtain an error reduction by by a factor of $\varepsilon$ one needs

$$
N = \frac{\log \varepsilon}{\log (1-1/\kappa)} \approx \kappa \, | \log \varepsilon |
$$

## Experiments with the Richardson iteration

In [ ]:
from ngsolve import *
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))

In [ ]:
fes = H1(mesh, order=1)
u,v = fes.TnT()
a = BilinearForm(grad(u)*grad(v)*dx+10*u*v*dx).Assemble()
f = LinearForm(x*y*v*dx).Assemble()
gfu = GridFunction(fes)

we determine (an approximation to) the largest eigenvalue by a few steps of the power iteration:

In [ ]:
hv = gfu.vec.CreateVector()
hv2 = gfu.vec.CreateVector()
hv.SetRandom()
hv.data /= Norm(hv)
for k in range(20):
    hv2.data = a.mat * hv
    rho = Norm(hv2)
    print (rho)
    hv.data = 1/rho * hv2

Run the Richardson iteration:

In [ ]:
alpha = 1 / rho
r = f.vec.CreateVector()
gfu.vec[:] = 0
err0 = Norm(f.vec)
its = 0
errhist = []
while True:
    r.data = f.vec - a.mat * gfu.vec
    err = Norm(r)
    print ("iteration", its, "res=", err)
    errhist.append (err)
    gfu.vec.data += alpha * r
    if err < 1e-8 * err0 or its > 10000: break
    its = its+1
print ("needed", its, "iterations")

In [ ]:
import matplotlib.pyplot as plt
plt.yscale('log')
plt.plot (errhist);

Experimenting with the mesh-size $h$, we observe experimentally that the number of iterations and thus the condition number grow proportional to $h^{-2}$. This can be easily calculated for a finite difference discretization on a uniform mesh. We will prove it for finite elements later.

In [ ]:
from ngsolve.webgui import Draw
Draw (gfu);

**Excercise:** 
* experiment with the relaxation parameter $\alpha$, how do the iteration numbers change ?
* halve the mesh-size, how do the iteration number change ?
* modify the parameter in front of the `u*v` term of the bilinear form. How does the iteration number change ? 